There are many dangerous bodies in space, one of them is N.E.O. - "Nearest Earth Objects". Some such bodies really pose a danger to the planet Earth, NASA classifies them as "is_hazardous". This dataset contains ALL NASA observations of similar objects from 1910 to 2024!!!

There are 338,199 records of N.E.O. in the Dataset!

Try to predict "is_hazardous" as accurately as possible! (otherwise we will not be ready for an asteroid attack)

In [5]:
# import libraries for classification
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier



In [6]:
df = pd.read_csv('../Data/NASA.csv')
print(df.head())
print(df.info())
print(df.describe())


    neo_id                name  absolute_magnitude  estimated_diameter_min  \
0  2162117  162117 (1998 SD15)               19.14                0.394962   
1  2349507    349507 (2008 QY)               18.50                0.530341   
2  2455415    455415 (2003 GA)               21.45                0.136319   
3  3132126           (2002 PB)               20.63                0.198863   
4  3557844           (2011 DW)               22.70                0.076658   

   estimated_diameter_max orbiting_body  relative_velocity  miss_distance  \
0                0.883161         Earth       71745.401048   5.814362e+07   
1                1.185878         Earth      109949.757148   5.580105e+07   
2                0.304818         Earth       24865.506798   6.720689e+07   
3                0.444672         Earth       78890.076805   3.039644e+07   
4                0.171412         Earth       56036.519484   6.311863e+07   

   is_hazardous  
0         False  
1          True  
2         Fals

In [12]:


# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

# Check for duplicates
print("Number of duplicates:")
print(df.duplicated().sum())



# Drop duplicates
df.drop_duplicates(inplace=True)

# Drop columns that are not needed

df.drop(['name', 'orbiting_body'], inplace=True, axis=1)

# fill missing values
def fill_missing_values(df, inplace=True):
    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = df[column].fillna(df[column].mode()[0])
        else:
            df[column] = df[column].fillna(df[column].mean())
    return df

df = fill_missing_values(df, inplace=True)

df.head(5)






Missing values per column:
neo_id                    0
absolute_magnitude        0
estimated_diameter_min    0
estimated_diameter_max    0
relative_velocity         0
miss_distance             0
is_hazardous              0
dtype: int64
Number of duplicates:
0


"# Drop duplicates\ndf.drop_duplicates(inplace=True)\n\n# Drop columns that are not needed\n\ndf.drop(['name', 'orbiting_body'], inplace=True, axis=1)\n\n# fill missing values\ndef fill_missing_values(df, inplace=True):\n    for column in df.columns:\n        if df[column].dtype == 'object':\n            df[column] = df[column].fillna(df[column].mode()[0])\n        else:\n            df[column] = df[column].fillna(df[column].mean())\n    return df\n\ndf = fill_missing_values(df, inplace=True)\n\ndf.head(5)"

In [8]:
# Outliers detection 

def detect_outliers(df, inplace=True):
    for column in df.columns:
        if df[column].dtype == 'int64' or df[column].dtype == 'float64':
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            df = df[(df[column] > lower_bound) & (df[column] < upper_bound)]
    return df 

df = detect_outliers(df, inplace=True)

df.head(5)


,neo_id,absolute_magnitude,estimated_diameter_min,estimated_diameter_max,relative_velocity,miss_distance,is_hazardous
2,2455415,21.45,0.136319,0.304818,24865.506798,6.720689e+07,False
3,3132126,20.63,0.198863,0.444672,78890.076805,3.039644e+07,False
4,3557844,22.70,0.076658,0.171412,56036.519484,6.311863e+07,False
5,3656926,25.00,0.026580,0.059435,47477.649832,4.290521e+07,False
6,3421513,21.50,0.133216,0.297879,57853.295346,2.727908e+07,True


In [13]:
# Data visualization

# Plotting the distribution of the target variable
plt.figure(figsize=(10, 6))
sns.countplot(df['is_hazardous'])
plt.title('Distribution of Hazardous')

# Plotting the distribution of the absolute magnitude

plt.figure(figsize=(10, 6))
sns.distplot(df['absolute_magnitude'], color='red')
plt.title('Distribution of Absolute Magnitude')


# Plotting the distribution of the estimated diameter min

plt.figure(figsize=(10, 6)) 
sns.distplot(df['estimated_diameter_min'], color='blue')
plt.title('Distribution of Estimated Diameter Min')

# Plotting the distribution of the estimated diameter max

plt.figure(figsize=(10, 6))
sns.distplot(df['estimated_diameter_max'], color='green')
plt.title('Distribution of Estimated Diameter Max')

# Plotting the distribution of the relative velocity

plt.figure(figsize=(10, 6))
sns.distplot(df['relative_velocity'], color='purple')
plt.title('Distribution of Relative Velocity')

# Plotting the distribution of the miss distance

plt.figure(figsize=(10, 6))
sns.distplot(df['miss_distance'], color='orange')
plt.title('Distribution of Miss Distance')






C:\Users\pieta\AppData\Local\Temp\ipykernel_27444\596019009.py:11: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(df['absolute_magnitude'], color='red')
C:\Users\pieta\AppData\Local\Temp\ipykernel_27444\596019009.py:18: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(df['estimated_diameter_min'], c

Text(0.5, 1.0, 'Distribution of Miss Distance')

Error in callback <function _draw_all_if_interactive at 0x000002435F090FE0> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

: 

: 

In [9]:
# categorical encoding

def encode_categorical(df, inplace=True):
    le = LabelEncoder()
    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = le.fit_transform(df[column])
    return df

df = encode_categorical(df, inplace=True)   

df.head(5)

,neo_id,absolute_magnitude,estimated_diameter_min,estimated_diameter_max,relative_velocity,miss_distance,is_hazardous
2,2455415,21.45,0.136319,0.304818,24865.506798,6.720689e+07,False
3,3132126,20.63,0.198863,0.444672,78890.076805,3.039644e+07,False
4,3557844,22.70,0.076658,0.171412,56036.519484,6.311863e+07,False
5,3656926,25.00,0.026580,0.059435,47477.649832,4.290521e+07,False
6,3421513,21.50,0.133216,0.297879,57853.295346,2.727908e+07,True


In [10]:
# scaling the data

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df.drop('is_hazardous', axis=1))
scaled_df = pd.DataFrame(scaled_data, columns=df.drop('is_hazardous', axis=1).columns)

print(scaled_df.head(5))


     neo_id  absolute_magnitude  estimated_diameter_min  \
0 -0.714808           -0.841515                0.553580   
1 -0.686030           -1.170353                1.290769   
2 -0.667925           -0.340238               -0.149618   
3 -0.663712            0.582112               -0.739860   
4 -0.673723           -0.821464                0.517006   

   estimated_diameter_max  relative_velocity  miss_distance  
0                0.553580          -0.991316       1.272666  
1                1.290769           1.360548      -0.484653  
2               -0.149618           0.365659       1.077494  
3               -0.739860          -0.006936       0.112512  
4                0.517006           0.444749      -0.633475  


In [11]:
# split the data into features and target

X = scaled_df
y = df['is_hazardous']

# split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train the model

model = RandomForestClassifier()

model.fit(X_train, y_train)

# make predictions

y_pred = model.predict(X_test)

# evaluate the model

print('Accuracy:', accuracy_score(y_test, y_pred))

print('Classification Report:', classification_report(y_test, y_pred))

print('Confusion Matrix:', confusion_matrix(y_test, y_pred))




Accuracy: 0.9816768086544962
Classification Report:               precision    recall  f1-score   support

       False       0.99      0.99      0.99     52775
        True       0.95      0.88      0.91      6385

    accuracy                           0.98     59160
   macro avg       0.97      0.94      0.95     59160
weighted avg       0.98      0.98      0.98     59160

Confusion Matrix: [[52463   312]
 [  772  5613]]
